In [16]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import tensorflow as tf
import os
print(tf.__version__)
#print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

1.14.0


In [17]:
data_train = pd.read_csv("../input/train.csv")
data = data_train.drop(['Survived'],axis=1)
target = data_train['Survived']
x_test = pd.read_csv("../input/test.csv")
#print(data_train['Parch'].dtype)
#print(target)
#print(data.shape)
#print(x_test.shape)

In [18]:
###checking the number of missing values
#print(data_train.isnull().sum())

In [19]:
### Handling missing data
###combining complete data to obtain average value
total_data = pd.concat([data, x_test])


## missing Age data
#get mean Age value per gender
male_mean_age = total_data[total_data['Sex']=='male']['Age'].mean()
female_mean_age = total_data[total_data['Sex']=='female']['Age'].mean()
#print(male_mean_age,female_mean_age)

#filling the NaN values
data_train.loc[(data_train['Sex']=='male') & (data_train['Age'].isnull()),'Age'] = male_mean_age
data_train.loc[(data_train['Sex']=='female') & (data_train['Age'].isnull()),'Age'] = female_mean_age

x_test.loc[(x_test['Sex']=='male') & (x_test['Age'].isnull()),'Age'] = male_mean_age
x_test.loc[(x_test['Sex']=='female') & (x_test['Age'].isnull()),'Age'] = female_mean_age


## missing Cabin data
### filling missing cabin values with NA
data_train['Cabin'] = data_train['Cabin'].fillna('Z')
x_test['Cabin'] = x_test['Cabin'].fillna('Z')


## missing Embarked data
### filling missing embarked values with 'X'
data_train['Embarked'] = data_train['Embarked'].fillna('S')
x_test['Embarked'] = x_test['Embarked'].fillna('S')


## missing Embarked data
### filling missing fare values with mean Fare value
avg_fare = total_data['Fare'].mean()
data_train['Fare'] = data_train['Fare'].fillna(avg_fare)

In [20]:
#data_train['Survived'].unique()
#def label_fix(label):
 #   if label==0:
  #      return 0
   # else:
    #    return 1
#data_train['Survived']=data_train['Survived'].apply(label_fix)
#from sklearn.model_selection import train_test_split
#x_train,x_test,y_train,y_test=train_test_split(data,target,test_size=0.2,random_state=101)
#x_train.columns
#print(data_train['Survived'])

In [21]:
## Add features to the bucket: 
### Define continuous list
CONTI_FEATURES  = ['PassengerId', 'Age', 'Fare']
### Define the categorical list
#CATE_FEATURES = ['Embarked']

In [22]:
continuous_features = [tf.feature_column.numeric_column(k) for k in CONTI_FEATURES]
Embarked = tf.feature_column.categorical_column_with_hash_bucket('Embarked', hash_bucket_size=3)
Sex = tf.feature_column.categorical_column_with_vocabulary_list('Sex',['female','male'])
Parch = tf.feature_column.categorical_column_with_identity('Parch', num_buckets=7)
Pclass = tf.feature_column.categorical_column_with_identity('Pclass', num_buckets=4)
SibSp = tf.feature_column.categorical_column_with_identity('SibSp', num_buckets=9)
Cabin = tf.feature_column.categorical_column_with_hash_bucket('Cabin', hash_bucket_size=100)

In [23]:
model = tf.estimator.LinearClassifier(
    n_classes = 2,
    feature_columns = [Pclass] + [Sex] + [Embarked] + [Parch] + [SibSp] + [Cabin] + continuous_features)    
    
#model=tf.estimator.LinearClassifier(n_classes=2, feature_columns=feat_cols)

In [24]:
FEATURES = ['Pclass', 'Sex', 'Embarked', 'SibSp','Cabin', 'Parch', 'PassengerId', 'Age', 'Fare']
LABEL = 'Survived'
def get_input_fn(datak, num_epochs=None, n_batch = 128, shuffle=True):
    return tf.estimator.inputs.pandas_input_fn(
       x=pd.DataFrame({k: datak[k].values for k in FEATURES}),
       y = pd.Series(datak[LABEL].values),
       batch_size=n_batch,   
       num_epochs=num_epochs,
       shuffle=shuffle)


#input_func=tf.estimator.inputs.pandas_input_fn(x=data,y=target,batch_size=100,num_epochs=1,shuffle=False)

In [25]:
model.train(input_fn=get_input_fn(data_train, num_epochs=None, n_batch=128, shuffle=False), steps=1000)
#model.train(input_fn=input_func,steps=1000)

In [26]:
score = model.evaluate(input_fn=get_input_fn(data_train, num_epochs=None, n_batch = 128, shuffle=False), steps=1000)
print(score)

{'accuracy': 0.84619534, 'accuracy_baseline': 0.6161094, 'auc': 0.8818428, 'auc_precision_recall': 0.8689232, 'average_loss': 0.39499107, 'label/mean': 0.38389063, 'loss': 50.558857, 'precision': 0.809665, 'prediction/mean': 0.3746263, 'recall': 0.78354836, 'global_step': 1000}


In [27]:
#### IMPLEMENTING ABOVE eg USING DNN
embarked_embd = tf.feature_column.indicator_column(categorical_column=Embarked)
cabin_embd = tf.feature_column.indicator_column(categorical_column=Cabin)
Sex_embd = tf.feature_column.indicator_column(categorical_column=Sex)
Parch_embd = tf.feature_column.indicator_column(categorical_column=Parch)
Pclass_embd = tf.feature_column.indicator_column(categorical_column=Pclass)
SibSp_embd = tf.feature_column.indicator_column(categorical_column=SibSp)




model1 = tf.estimator.DNNClassifier(
    feature_columns = [Pclass_embd] + [Sex_embd] + [embarked_embd] + [Parch_embd] + [SibSp_embd] + [cabin_embd] + continuous_features,
    hidden_units = [64, 64])

model1.train(input_fn=get_input_fn(data_train, num_epochs=None, n_batch=128, shuffle=False), steps=1000)

In [28]:
score = model1.evaluate(input_fn=get_input_fn(data_train, num_epochs=None, n_batch = 128, shuffle=False), steps=1000)
print(score)

{'accuracy': 0.8418047, 'accuracy_baseline': 0.6161094, 'auc': 0.8971697, 'auc_precision_recall': 0.88083225, 'average_loss': 0.38066798, 'label/mean': 0.38389063, 'loss': 48.725502, 'precision': 0.80019534, 'prediction/mean': 0.40363425, 'recall': 0.78356874, 'global_step': 1000}
